In [1]:
import argparse
import base64
import json
import numpy as np
import time
import eventlet
import eventlet.wsgi
import tensorflow as tf
import socketio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from PIL import Image
from PIL import ImageOps
from flask import Flask, render_template
from io import BytesIO
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array

import numpy as np
import random
import math
from matplotlib import pyplot as plt

from numpy import array
import random
import scipy.misc

Using TensorFlow backend.


In [2]:
import PIL
import numpy
from PIL import Image

def resize_image_to_square(numpy_array_image, new_size):
    """
    I am NOT zero-padding at this moment, 
    just resizing for the longest size is equal to new_size.
    The zero-padding can effectively by done later,
    for example during machine learning.
    There is no point of wasting space with
    thens of thousands padded padded images. 
    """
    # convert nympy array image to PIL.Image
    image = Image.fromarray(numpy.uint8(numpy_array_image))
    old_width = float(image.size[0])
    old_height = float(image.size[1])
    
    if old_width > old_height:
        # by width since it is longer
        new_width = new_size
        ratio = float(new_width / old_width)
        new_height = int(old_height * ratio)
    else:
        # by height since it is longer
        new_width = new_size
        ratio = float(new_width / old_width)
        new_height = int(old_height * ratio)
        
    image = image.resize((new_width, new_height), PIL.Image.ANTIALIAS)
    # turn image into nympy array again
    return array(image)

In [3]:
def mask_vertices(image):
    """

    """
    height = image.shape[0]
    width = image.shape[1]

    top_left = (width*0.3, 0)
    top_right = (width-width*0.3, 0)
     
    mid_left_high = (0, height*0.2) 
    mid_right_high = (width, height*0.2)  
    
    mid_left_low = (0, height*0.9) 
    mid_right_low = (width, height*0.9)
    
    # on the bottom start high because of the dashboard
    bottom_center_left = (width*0.27, height*0.95) 
    bottom_center_right = (width-width*0.27, height*0.95) 
    
    # we are NOT following a center line in this code, so cut it out
    bottom_center = (width/2, height*0.55) 


    # add points clockwise
    vertices = np.array([[
        top_left, 
        top_right, 
        mid_right_high, mid_right_low,
        bottom_center_right,
        bottom_center, bottom_center_left,
        mid_left_low, 
        mid_left_high 
    ]], dtype=np.int32)
    return vertices

In [4]:
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [5]:
def canny(image, low_threshold=50, high_threshold=250): 
    # homework low_threshold=20, high_threshold=130
    """Applies the Canny transform"""
    return cv2.Canny(image, low_threshold, high_threshold)

In [6]:
def gaussian_blur(image, kernel_size=5): # 5 
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

In [7]:
import cv2
def grayscale(image):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [8]:
def crop_image(image):
    left = 0
    upper = 70
    right = 320
    lower = 140 # 160 original
    image = image.crop((left, upper, right, lower))
    return image

In [9]:
def preprocessing_pipline(image, final_size=224, should_plot=False):
    """
    final_size=256 AlexNet and GoogLeNet
    final_size=224 VGG-16
    final_size=64  is OPTIMAL if I was writing CDNN from scratch
    final_size=32  images are fuzzy, AlexNet (street signs CDNN)
    final_size=28  images are very fuzzy, LeNet
    """
    import matplotlib.pyplot as plt
    
    if should_plot:
        plt.imshow(image)
        plt.show()
        
    image = array(crop_image(image)) 
    # convert output to numpy array
    if should_plot:
        plt.imshow(image)
        plt.show()

    #image = region_of_interest(image, mask_vertices(image))
    
    image = grayscale(image)
    
    if should_plot:
        plt.imshow(image, cmap='gray')
        plt.show()

    image = gaussian_blur(image, kernel_size=5)
    
    if should_plot:
        plt.imshow(image, cmap='gray')
        plt.show()
    
    image = canny(image, low_threshold=100, high_threshold=190)
    
    if should_plot:
        print("image before resizing", image.shape)
        plt.imshow(image, cmap='gray')
        plt.show()
    
    image = resize_image_to_square(image, new_size=final_size) # VGG-16 size
    #image = cv2.resize(image,(224,224))
        
    if should_plot:
        print("image after resizing", image.shape)
        plt.imshow(image, cmap='gray')
        plt.show()
    return image